In [9]:
%pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd

df = pd.read_csv('data/customer_support_tickets.csv')

df.head()



,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [11]:
colunas_uteis = ['Ticket Description', 'Ticket Priority', 'Time to Resolution']
df_limpo = df[colunas_uteis].copy()

df_limpo.head()

,Ticket Description,Ticket Priority,Time to Resolution
0,I'm having an issue with the {product_purchase...,Critical,NaN
1,I'm having an issue with the {product_purchase...,Critical,NaN
2,I'm facing a problem with my {product_purchase...,Low,2023-06-01 18:05:38
3,I'm having an issue with the {product_purchase...,Low,2023-06-01 01:57:40
4,I'm having an issue with the {product_purchase...,Low,2023-06-01 19:53:42


In [12]:
print("Buracos antes da limpeza:")
print(df_limpo.isnull().sum())

df_limpo = df_limpo.dropna()

print("\nDados prontos para uso:", df_limpo.shape)

Buracos antes da limpeza:
Ticket Description       0
Ticket Priority          0
Time to Resolution    5700
dtype: int64

Dados prontos para uso: (2769, 3)


In [13]:
print(df_limpo['Ticket Priority'].value_counts())

Ticket Priority
Critical    726
High        705
Medium      694
Low         644
Name: count, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Separando X (texto) e y (alvo)
X = df_limpo['Ticket Description']
y = df_limpo['Ticket Priority']

# 2. Dividindo em Treino e Teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Criando o "Tradutor" (Vetorizador)
# Aaprendendo o vocabulário apenas com os dados de treino
vetorizador = TfidfVectorizer(stop_words='english', max_features=1000)
X_train_vetorizado = vetorizador.fit_transform(X_train)
X_test_vetorizado = vetorizador.transform(X_test)

# 4. Criando e Treinando o Modelo
modelo = LogisticRegression()
modelo.fit(X_train_vetorizado, y_train)

# 5. Avaliando
previsoes = modelo.predict(X_test_vetorizado)
acuracia = accuracy_score(y_test, previsoes)

print(f"Acurácia do Modelo: {acuracia:.2%}")
print("\nRelatório Detalhado:\n", classification_report(y_test, previsoes))

# 6. Salvando os arquivos para usar na API
joblib.dump(modelo, 'modelo_prioridade.pkl')
joblib.dump(vetorizador, 'vetorizador.pkl')
print("Arquivos .pkl salvos com sucesso!")

Acurácia do Modelo: 26.71%

Relatório Detalhado:
               precision    recall  f1-score   support

    Critical       0.31      0.28      0.29       156
        High       0.24      0.29      0.27       130
         Low       0.22      0.18      0.20       133
      Medium       0.28      0.31      0.30       135

    accuracy                           0.27       554
   macro avg       0.26      0.27      0.26       554
weighted avg       0.27      0.27      0.27       554

Arquivos .pkl salvos com sucesso!
